<a href="https://kritikseth.github.io/ipynbtagredirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/assets/icons/kritik_ipynbtagredirect.svg" alt="Kritik Seth"/></a>

## Behavior of the urban traffic of the city of Sao Paulo in Brazil Data Set
By- [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.html)


[Click Here](https://archive.ics.uci.edu/ml/datasets/Behavior+of+the+urban+traffic+of+the+city+of+Sao+Paulo+in+Brazil) to view the dataset page

In [1]:
!wget -O 'BehaviorUTBrazil.zip' 'https://archive.ics.uci.edu/ml/machine-learning-databases/00483/Behavior%20of%20the%20urban%20traffic%20of%20the%20city%20of%20Sao%20Paulo%20in%20Brazil.zip'
!mkdir BehaviorUTBrazil
!unzip BehaviorUTBrazil.zip -d /content/BehaviorUTBrazil

--2020-09-21 12:52:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/00483/Behavior%20of%20the%20urban%20traffic%20of%20the%20city%20of%20Sao%20Paulo%20in%20Brazil.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638858 (624K) [application/x-httpd-php]
Saving to: ‘BehaviorUTBrazil.zip’

BehaviorUTBrazil.zi 100%[===================>] 623.88K   827KB/s    in 0.8s    

2020-09-21 12:52:46 (827 KB/s) - ‘BehaviorUTBrazil.zip’ saved [638858/638858]

Archive:  BehaviorUTBrazil.zip
   creating: /content/BehaviorUTBrazil/Behavior of the urban traffic of the city of Sao Paulo in Brazil/
  inflating: /content/BehaviorUTBrazil/Behavior of the urban traffic of the city of Sao Paulo in Brazil/Behavior of the urban traffic of the city of Sao Paulo in Brazil.csv  
  inflating: /content/BehaviorUTBrazil/Behavior of the ur

In [2]:
import numpy as np
import pandas as pd
from scipy.io import arff
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go

In [3]:
col_names = ['Hour', 'Immobilized Bus', 'Broken Truck', 'Vehice Excess', 'Accident Victom',
             'Running Over', 'Fire Vehicles', 'Occurance Involving Freight', 'Incident Involving Dabgerous Freight',
             'Lack of Electricity', 'Fire', 'Point of Flooding', 'Manifestations', 'Defect in the Network of Trolley Buses',
             'Tree on Road', 'Semaphore off', 'Intermittent Semaphore', 'Slowness in Traffic (%)']

In [4]:
data = arff.loadarff('/content/BehaviorUTBrazil/Behavior of the urban traffic of the city of Sao Paulo in Brazil/Behavior of the urban traffic.arff')
data = [list(i) for i in data[0]]
data = pd.DataFrame(data, columns=col_names)

In [5]:
data.head()

,Hour,Immobilized Bus,Broken Truck,Vehice Excess,Accident Victom,Running Over,Fire Vehicles,Occurance Involving Freight,Incident Involving Dabgerous Freight,Lack of Electricity,Fire,Point of Flooding,Manifestations,Defect in the Network of Trolley Buses,Tree on Road,Semaphore off,Intermittent Semaphore,Slowness in Traffic (%)
0,b'7:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1
1,b'7:30',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.6
2,b'8:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.7
3,b'8:30',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.2
4,b'9:00',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.1


In [6]:
data['Hour'] = data['Hour'].astype(str)
data['Hour'] = data['Hour'].apply(lambda x: x.split('\'')[1])
data['Hour'] = data['Hour'].apply(lambda x: int(x.split(':')[0])*60 + int(x.split(':')[1]))
data['Hour'] = data['Hour']/60

data['Slowness in Traffic (%)'] = data['Slowness in Traffic (%)']/100

In [7]:
data.head()

,Hour,Immobilized Bus,Broken Truck,Vehice Excess,Accident Victom,Running Over,Fire Vehicles,Occurance Involving Freight,Incident Involving Dabgerous Freight,Lack of Electricity,Fire,Point of Flooding,Manifestations,Defect in the Network of Trolley Buses,Tree on Road,Semaphore off,Intermittent Semaphore,Slowness in Traffic (%)
0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041
1,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066
2,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087
3,8.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092
4,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111


In [8]:
for i in range(1,17):
    data.iloc[:,i] = data.iloc[:,i].astype(int)

In [9]:
X = data.drop(['Slowness in Traffic (%)'], axis=1)
Y = data['Slowness in Traffic (%)']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [12]:
X_train.shape, X_test.shape, X_val.shape

((81, 17), (27, 17), (27, 17))

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import learning_curve

In [14]:
def cv(model, modelName,data):
    X, Y = data
    scores = cross_val_score(model, X, Y, cv=5)
    print('\n' + '_'*50)
    print(f'\n{modelName}\n\tBest Score:    {max(scores)}\n\tAverage Score: {np.mean(scores)}\n\tStd Dev:       {np.std(scores)}')

In [15]:
rfr = RandomForestRegressor(n_estimators=100)
lasso = Lasso(alpha=0.01)
ridge = Ridge(alpha=0.01)
linear = LinearRegression()

In [16]:
model = [rfr, lasso, ridge, linear]
modelNames = ['RandomForestRegressor', 'Lasso', 'Ridge', 'LinearRegression']

for i in tqdm(range(len(model))):
    assert(len(model)==len(modelNames))
    cv(model[i], modelNames[i], (X_val, y_val))

100%|██████████| 4/4 [00:00<00:00,  5.43it/s]


__________________________________________________

RandomForestRegressor
	Best Score:    0.8089835585419728
	Average Score: 0.5676117399939324
	Std Dev:       0.22056080262873218

__________________________________________________

Lasso
	Best Score:    0.7169891352566533
	Average Score: 0.5168346953295588
	Std Dev:       0.19302478926428215

__________________________________________________

Ridge
	Best Score:    0.3615501585359061
	Average Score: 0.012962040504341887
	Std Dev:       0.29932173693927483

__________________________________________________

LinearRegression
	Best Score:    0.3501344515457169
	Average Score: -0.02128603869847632
	Std Dev:       0.3279966347565938


In [17]:
 param_grid = {'n_estimators': [10, 20, 30, 40, 50],
               'max_depth': [5, 10, 15],
               'min_samples_split': [10, 20],
               'min_samples_leaf': [5, 10, 20],
               'max_leaf_nodes': [20, 40],
               'min_weight_fraction_leaf': [0.1]}

In [18]:
grid_search = GridSearchCV(rfr, param_grid=param_grid, cv=5)

In [19]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [20]:
grid_search.best_params_

{'max_depth': 15,
 'max_leaf_nodes': 20,
 'min_samples_leaf': 5,
 'min_samples_split': 20,
 'min_weight_fraction_leaf': 0.1,
 'n_estimators': 10}

In [21]:
grid_search.best_score_

0.41513643819397206

In [22]:
rfr = RandomForestRegressor(**grid_search.best_params_)

In [23]:
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=15, max_features='auto', max_leaf_nodes=20,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=20, min_weight_fraction_leaf=0.1,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [24]:
y_pred = rfr.predict(X_test)

In [25]:
acc = rfr.score(X_test, y_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'Accuracy: {acc}\nRMSE: {rmse}')

Accuracy: 0.5453801065457116
RMSE: 0.03106288738528612


In [26]:
fig = px.histogram(histfunc='sum', y=rfr.feature_importances_, x=X.columns)
fig.show()

In [27]:
train_sizes, train_scores, valid_scores = learning_curve(rfr, X_train, y_train, cv=5)

In [28]:
fig = px.line(x=train_sizes, y=train_scores[0], title='Training Scores')
fig.show()